In [1]:
!pip install requests transformers pandas datetime vaderSentiment requests yfinance finnhub-python
!pip install nltk
!pip install vaderSentiment
!pip install yahoo-fin
!pip install finvizfinance
!pip install "websockets>=11.0"
!pip install yfinance yahoo_fin nltk textblob matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 8.1 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=986c23f3286368d42953ee152e31b157ec415b8743cf82919005c3871ce4ef2c
  Stored

In [6]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import pytz
from yahoo_fin import news as yahoo_fin_news
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# Setup
nltk.download('vader_lexicon')
vader = SentimentIntensityAnalyzer()
tz = pytz.timezone('US/Eastern')
now = datetime.now(tz)
start_window = now - timedelta(days=90)

from google.colab import files
uploaded = files.upload()
with open("company_tickers.txt", "r") as f:
    content = f.read()

TICKERS = [ticker.strip().replace('"', '') for ticker in content.split(',')]

INTERVALS = {
    "1 Hour Ago": timedelta(hours=1),
    "4 Hours Ago": timedelta(hours=4),
    "Previous Day Close": timedelta(days=1),
    "7 Days Ago": timedelta(days=7)
}

class NewsAnalyzer:
    def __init__(self, ticker):
        self.ticker = ticker
        self.stock = yf.Ticker(ticker)

    def fetch_articles(self):
        try:
            all_articles = [
                {
                    "Ticker": self.ticker,
                    "Title": art["title"],
                    "Summary": art["summary"],
                    "URL": art["link"],
                    "Datetime": datetime.strptime(art["published"], "%a, %d %b %Y %H:%M:%S %z").astimezone(tz)
                }
                for art in yahoo_fin_news.get_yf_rss(self.ticker)
                if start_window <= datetime.strptime(art["published"], "%a, %d %b %Y %H:%M:%S %z").astimezone(tz) <= now
            ]
            # Sort by datetime (most recent first), and take top 3
            all_articles.sort(key=lambda x: x["Datetime"], reverse=True)
            return all_articles[:3]
        except Exception as e:
            print(f"Error fetching news for {self.ticker}: {e}")
            return []

    def get_price_near(self, timepoint):
        try:
            hist = self.stock.history(
                start=timepoint.strftime('%Y-%m-%d'),
                end=(timepoint + timedelta(days=1)).strftime('%Y-%m-%d'),
                interval='1h'
            )
            if not hist.empty:
                hist['Delta'] = abs(hist.index - timepoint)
                return round(hist.sort_values('Delta').iloc[0]['Close'], 2)
        except:
            return None
        return None

    def analyze_article(self, article):
        base_time = article["Datetime"]
        base_price = self.get_price_near(base_time)
        if base_price is None:
            return None

        prices = {}
        pct_changes = {}
        for label, delta in INTERVALS.items():
            time_check = base_time - delta
            past_price = self.get_price_near(time_check)
            prices[label] = past_price if past_price else 'N/A'
            pct_changes[label + " % Change"] = (
                round(((base_price - past_price) / past_price) * 100, 2) if isinstance(past_price, (int, float)) else 'N/A'
            )

        return {
            **article,
            "News Age (Days)": (now - base_time).days,
            "News Time": base_time,
            "Current Price": base_price,
            **prices,
            **pct_changes,
            "Headline Sentiment": round(TextBlob(article["Title"]).sentiment.polarity, 2),
            "Summary Sentiment": round(vader.polarity_scores(article["Summary"])["compound"], 2)
        }

def run_full_analysis():
    full_dataset = []
    for ticker in TICKERS:
        analyzer = NewsAnalyzer(ticker)
        articles = analyzer.fetch_articles()
        for article in articles:
            result = analyzer.analyze_article(article)
            if result:
                full_dataset.append(result)

    df = pd.DataFrame(full_dataset)
    df.to_csv("Companies_Sentiment_Analysis_7_Days_Track.csv", index=False)
    print("Saved to Companies_Sentiment_Analysis_7_Days_Track.csv")
    return df

# Run it
df_result = run_full_analysis()
print(df_result.head())

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Saving company_tickers.txt to company_tickers (3).txt


ERROR:yfinance:$CSCO: possibly delisted; no price data found  (1h 2025-08-04 -> 2025-08-05)
ERROR:yfinance:$CSCO: possibly delisted; no price data found  (1h 2025-08-04 -> 2025-08-05)
ERROR:yfinance:$CSCO: possibly delisted; no price data found  (1h 2025-08-04 -> 2025-08-05)
ERROR:yfinance:$NET: possibly delisted; no price data found  (1h 2025-08-04 -> 2025-08-05)
ERROR:yfinance:$HMC: possibly delisted; no price data found  (1h 2025-08-04 -> 2025-08-05)
ERROR:yfinance:$HMC: possibly delisted; no price data found  (1h 2025-08-04 -> 2025-08-05)
ERROR:yfinance:$HMC: possibly delisted; no price data found  (1h 2025-08-04 -> 2025-08-05)
ERROR:yfinance:$CRM: possibly delisted; no price data found  (1h 2025-08-02 -> 2025-08-03)
ERROR:yfinance:$CRM: possibly delisted; no price data found  (1h 2025-08-02 -> 2025-08-03)
ERROR:yfinance:$RIVN: possibly delisted; no price data found  (1h 2025-08-04 -> 2025-08-05)
ERROR:yfinance:$RIVN: possibly delisted; no price data found  (1h 2025-08-04 -> 2025-0

Saved to Companies_Sentiment_Analysis_7_Days_Track.csv
  Ticker                                              Title  \
0    NET  Cloudflare (NET) Reports Increased Revenue Ami...   
1    NET  Cloudflare Q2 Earnings and Revenues Beat Estim...   
2    CRM  CRM vs. NOW: Which Workflow Automation Stock H...   
3   PANW  Palo Alto Networks (PANW) to Acquire CyberArk ...   
4   PANW  SentinelOne (S) Gains High-Level CCN Certifica...   

                                             Summary  \
0  Cloudflare (NET) recently announced significan...   
1  NET beats Q2 earnings and revenue estimates, d...   
2  ServiceNow and Salesforce tap rising demand fo...   
3  Palo Alto Networks, Inc. (NASDAQ:PANW) is one ...   
4  SentinelOne (S) recently achieved high-level c...   

                                                 URL  \
0  https://finance.yahoo.com/news/cloudflare-net-...   
1  https://finance.yahoo.com/news/cloudflare-q2-e...   
2  https://finance.yahoo.com/news/crm-vs-now-work...   
3  ht

In [7]:
from google.colab import files
files.download("Companies_Sentiment_Analysis_7_Days_Track.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>